In [1]:


%%capture
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
import os
import sys
from dotenv import load_dotenv
from pathlib import Path
import datetime as d 
load_dotenv()

# get the current directory
current_dir = os.getcwd()

# change the current working directory to the parent directory
os.chdir(os.path.dirname(current_dir))

path = os.getcwd()
parent = os.path.abspath(os.path.join(path, os.pardir))

USRPATH = "/Users/ducjeremyvu/dev/"


sys.path.append(Path().absolute().parent)
sys.path.append("/Users/ducjeremyvu/dev/wspace/nb")
sys.path.append(parent)
sys.path.append(...)

os.getcwd()


# from g_api import gservice


In [2]:
YEAR = 2024
MONTH = 11

# Step 1: get bonus data and get contact data and merge and create file 

how to get the bonus data: 

https://docs.google.com/spreadsheets/d/1434XUVuzcmaEu7WtnRImQO9_uy0NQp27PB9geS79uhA/edit?resourcekey#gid=627741291


there you can find all relevant bonus data

Bonus_raw_template


then copy the sheet by creating a new sheet (there is a right click function for that)

then change the locale to US or UK 
(reason for this is to change floats with commas to dot notation, due to **comma** seperated values (csv) download, which conflict with germans numeric representation)

then download csv and add the path to next cell

In [3]:
path_bonus = "/Users/ducjeremyvu/Downloads/ucmdownload/241209_manual_corrections - Bonus_raw_template.csv"

data_bonus = pd.read_csv(path_bonus)

# create list of students that get bonus to get the max sequence number 
list_bonus_user_ids = data_bonus["user_id"].to_list()
#string_list_bonus_user_ids = ",".join(list_bonus_user_ids)  #make a string list with commas for the ids 

data_bonus

,user_id,VD_EBETRAG,VD_TXT,PNR
0,80672,50.0,Smartseller Nürnberg Bonus,A80672
1,78357,50.0,Smartseller Nürnberg Bonus,A78357
2,71097,200.0,Heinemann DUS,A71097
3,70959,200.0,Heinemann DUS,A70959
4,73363,200.0,Heinemann DUS,A73363
5,71523,200.0,Heinemann DUS,A71523
6,70696,300.0,Heinemann DUS,A70696
7,73428,200.0,Heinemann BER,A73428
8,72035,50.0,Heinemann DUS,A72035
9,75551,200.0,Bonus,A75551


In [4]:
from ucmdata.database.psql import Postgres
from sshtunnel import SSHTunnelForwarder

server = SSHTunnelForwarder(("djvsrv1.netbird.cloud", 22),
         ssh_username="djv",
         ssh_password="ucmsysadmin",
         remote_bind_address=('ucm-main-read-replica-public.c03qb56spoj7.eu-central-1.rds.amazonaws.com', 5432),
         local_bind_address=('localhost', 60000))
        
server.start()

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.9p1)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.9p1)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.9p1)
INFO:paramiko.transport:Authentication (password) successful!


In [5]:
from ucmdata.database.psql import Postgres 

pg = Postgres(port=60000, host="localhost")
# params = {"user_ids":}

# Dynamically create placeholders based on the number of user_ids
placeholders = ', '.join(['%s'] * len(list_bonus_user_ids))

query= f"""
    SELECT
        ja.user_id,
        max(ujd.sequence),
        max(ujd.checkin_time)::date as max_date
    FROM
        user_job_dates AS ujd
        JOIN job_applications AS ja ON ja.id = ujd.job_application_id
    where 
        ja.user_id in ({placeholders})
        and ujd.sequence is not null
    GROUP BY
        ja.user_id;
    
"""

data_sequence = pg.get_data(query = query, params=list_bonus_user_ids).dropna()

data_sequence["VERTNR"] = data_sequence["max"].astype(int)
data_sequence["VD_ZDAT"] = pd.to_datetime(data_sequence["max_date"]).apply(lambda x: x.replace(day=1))


# data_sequence.to_excel("/Users/ducjeremyvu/Downloads/ucmdownload/1.xlsx")

data_sequence

  self._data = read_sql_query(query, conn, params=params)



,user_id,max,max_date,VERTNR,VD_ZDAT
0,70696,3,2024-11-16,3,2024-11-01
1,70959,2,2024-11-30,2,2024-11-01
2,71097,3,2024-11-29,3,2024-11-01
3,71523,2,2024-11-30,2,2024-11-01
4,72035,2,2024-11-23,2,2024-11-01
5,73334,2,2024-11-14,2,2024-11-01
6,73363,2,2024-10-14,2,2024-10-01
7,73428,1,2024-10-26,1,2024-10-01
8,75551,1,2024-10-31,1,2024-10-01
9,76341,1,2024-11-22,1,2024-11-01


In [6]:
server.stop()

In [7]:
data_wd_input = data_bonus.merge(data_sequence, on="user_id")

data_wd_input["LA"] = "743"
data_wd_input["AK"] = "201"

data_wd_input

,user_id,VD_EBETRAG,VD_TXT,PNR,max,max_date,VERTNR,VD_ZDAT,LA,AK
0,80672,50.0,Smartseller Nürnberg Bonus,A80672,3,2024-11-20,3,2024-11-01,743,201
1,78357,50.0,Smartseller Nürnberg Bonus,A78357,3,2024-11-20,3,2024-11-01,743,201
2,71097,200.0,Heinemann DUS,A71097,3,2024-11-29,3,2024-11-01,743,201
3,70959,200.0,Heinemann DUS,A70959,2,2024-11-30,2,2024-11-01,743,201
4,73363,200.0,Heinemann DUS,A73363,2,2024-10-14,2,2024-10-01,743,201
5,71523,200.0,Heinemann DUS,A71523,2,2024-11-30,2,2024-11-01,743,201
6,70696,300.0,Heinemann DUS,A70696,3,2024-11-16,3,2024-11-01,743,201
7,73428,200.0,Heinemann BER,A73428,1,2024-10-26,1,2024-10-01,743,201
8,72035,50.0,Heinemann DUS,A72035,2,2024-11-23,2,2024-11-01,743,201
9,75551,200.0,Bonus,A75551,1,2024-10-31,1,2024-10-01,743,201


In [8]:
wd_column_order = [
    "SATZART",
    "FUNKTION",
    "MAN",
    "AK",
    "PNR",
    "NAME",
    "VORNAME",
    "VERTNR",
    "LA",
    "VD_ETAGE",
    "VD_ESTD",
    "VD_EFAKT",
    "VD_EBETRAG",
    "KST",
    "KOSTART",    
    "VD_KTR",
    "VD_DAT",
    "VD_ZDAT",
    "VD_HER",
    "VD_HER_DATE",
    "VD_PROZ",
    "VD_KSTB",
    "VD_USERID",
    "VD_WERT",
    "KST2_MAN",
    "KST2_AK",
    "VD_KALK",
    "VD_TXT",
]

def fill_var(dataframe, YEAR, MONTH):
    MONTH = str(MONTH) if len(str(MONTH)) == 2 else "0" + str(MONTH)
    YEAR_SHORT = str(YEAR)[2:4]
    dataframe = dataframe.copy()
    dataframe["MAN"] = "10246"
    dataframe["SATZART"] = "[VARTAB]"
    dataframe["FUNKTION"] = "INSERT"
#    dataframe["LA"] = LA
    dataframe["VD_HER"] = f"2C{YEAR_SHORT}{MONTH}"
    # dataframe["VD_ZDAT"] = f"{YEAR}-{MONTH}-01"
    dataframe["VD_DAT"] = ""
    dataframe["VD_ETAGE"] = ""
    dataframe["VD_EBETRAG"] = dataframe["VD_EBETRAG"].astype(str).str.replace(".",",")
    dataframe["KST"] = "1100"
    dataframe["KOSTART"] = ""
    dataframe["VD_KTR"] = ""
    dataframe["VD_ESTD"] = ""
    dataframe["VD_EFAKT"] = ""
    dataframe["NAME"] = ""
    dataframe["VORNAME"] = "" 
    dataframe["VD_HER_DATE"] = ""
    dataframe["VD_PROZ"] = ""
    dataframe["VD_KSTB"] = ""
    dataframe["VD_USERID"] = ""
    dataframe["VD_WERT"] = ""
    dataframe["KST2_MAN"] = ""
    dataframe["KST2_AK"] = ""
    dataframe["VD_KALK"] = ""
#    dataframe["VD_TXT"] = dataframe["text"]



    return dataframe[wd_column_order]

In [9]:
pd.options.display.max_columns = 100



output = fill_var(data_wd_input, YEAR, MONTH)

output

,SATZART,FUNKTION,MAN,AK,PNR,NAME,VORNAME,VERTNR,LA,VD_ETAGE,VD_ESTD,VD_EFAKT,VD_EBETRAG,KST,KOSTART,VD_KTR,VD_DAT,VD_ZDAT,VD_HER,VD_HER_DATE,VD_PROZ,VD_KSTB,VD_USERID,VD_WERT,KST2_MAN,KST2_AK,VD_KALK,VD_TXT
0,[VARTAB],INSERT,10246,201,A80672,,,3,743,,,,"50,0",1100,,,,2024-11-01,2C2411,,,,,,,,,Smartseller Nürnberg Bonus
1,[VARTAB],INSERT,10246,201,A78357,,,3,743,,,,"50,0",1100,,,,2024-11-01,2C2411,,,,,,,,,Smartseller Nürnberg Bonus
2,[VARTAB],INSERT,10246,201,A71097,,,3,743,,,,"200,0",1100,,,,2024-11-01,2C2411,,,,,,,,,Heinemann DUS
3,[VARTAB],INSERT,10246,201,A70959,,,2,743,,,,"200,0",1100,,,,2024-11-01,2C2411,,,,,,,,,Heinemann DUS
4,[VARTAB],INSERT,10246,201,A73363,,,2,743,,,,"200,0",1100,,,,2024-10-01,2C2411,,,,,,,,,Heinemann DUS
5,[VARTAB],INSERT,10246,201,A71523,,,2,743,,,,"200,0",1100,,,,2024-11-01,2C2411,,,,,,,,,Heinemann DUS
6,[VARTAB],INSERT,10246,201,A70696,,,3,743,,,,"300,0",1100,,,,2024-11-01,2C2411,,,,,,,,,Heinemann DUS
7,[VARTAB],INSERT,10246,201,A73428,,,1,743,,,,"200,0",1100,,,,2024-10-01,2C2411,,,,,,,,,Heinemann BER
8,[VARTAB],INSERT,10246,201,A72035,,,2,743,,,,"50,0",1100,,,,2024-11-01,2C2411,,,,,,,,,Heinemann DUS
9,[VARTAB],INSERT,10246,201,A75551,,,1,743,,,,"200,0",1100,,,,2024-10-01,2C2411,,,,,,,,,Bonus


In [10]:
filename = f"{YEAR}_{MONTH}_student_bonus_manual_student_service"

output.to_csv("/Users/ducjeremyvu/Downloads/ucmdownload/" + filename + ".csv", index=False, sep=";")